In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/d/luyangshang/finance/data.csv


In [17]:
import argparse
import pandas as pd
from scipy.stats.mstats import winsorize
import lightgbm as lgb
import numpy as np

def harness(data_path):

    # Load dataset
    df = pd.read_csv(data_path)

    # transform column
    df['current_asst_liability_ratio'] = df['asst_current']/(df['asst_tot']-df['eqty_tot'])
    df['current_ratio_st'] = df['asst_current']/(df['debt_bank_st'] + df['debt_fin_st'] + df['AP_st'] + df['debt_st'])
    df['intangible_asst_ratio'] = df['asst_intang_fixed']/df['asst_tot']
    df['current_asst_ratio'] = df['asst_current']/df['asst_tot']
    df['cash_liability_ratio'] = df['cf_operations']/(df['debt_bank_st'] + df['debt_fin_st'] + df['AP_st'] + df['debt_st'])
    df['capital_employed'] = df['asst_tot'] - (df['debt_bank_st'] + df['debt_fin_st'] + df['AP_st'] + df['debt_st'])
    df['capital_turnover_ratio'] = (df['rev_operating']-df['COGS'])/df['capital_employed']
    df['ROCE'] = df['ebitda']/df['capital_employed']
    df['log_total_asset']=np.log(df['asst_tot'])
    df['debt_coverage_ratio'] = (df['ebitda'])/(df['debt_st'])
    df['leverage'] = (df['asst_tot'] - df['eqty_tot']) / df['eqty_tot']
    df['interest_coverage_ratio']=df['ebitda']/df['exp_financing']
    df['extraord_ratio'] = df['inc_extraord']/ df['rev_operating']
    df['finance_rev_ratio'] = df['inc_financing']/df['rev_operating']
    df['equity_asst_ratio'] = df['eqty_tot']/df['asst_tot']
    df['cash_asst_ratio'] = df['cf_operations']/df['asst_tot']

    # remove irrelavent columns
    df = df[['current_ratio_st',
 'roa',
 'cash_asst_ratio',
 'current_asst_ratio',
 'current_asst_liability_ratio',
 'debt_coverage_ratio',
 'intangible_asst_ratio',
 'capital_turnover_ratio',
 'equity_asst_ratio',
 'roe',
 'ROCE',
 'leverage',
 'log_total_asset',
 'cash_liability_ratio',
 'interest_coverage_ratio',
 'finance_rev_ratio',
 'capital_employed',
 'extraord_ratio']]

    # winsorize and fillna
    for col in df.columns:
        df[col] = pd.Series(winsorize(df[col],limits=[0.2,0.2],nan_policy='omit')) 
        df[col] = df[col].fillna(df[col].mean())

    model = lgb.Booster(model_file='../input/lgb-paramter/lgb_classifier.txt')

    pd.DataFrame(model.predict(df),columns=['default_prob']).to_csv('output.csv')

In [18]:
harness('../input/full-finance-dataset/train.csv')

In [10]:
df = pd.read_csv('../input/full-finance-dataset/train.csv')
# create default variable
def defaulted(ser):
    if pd.isnull(ser['def_date']) or (ser['def_date'] - ser['stmt_date']).days > 365:
        return 0 
    else:
        return 1

df['stmt_date'] = pd.to_datetime(df['stmt_date'])
df['def_date'] = pd.to_datetime(df['def_date'])
df['defaulted'] = df.apply(defaulted,axis=1)

In [19]:
roc_auc_score(df['defaulted'],pd.read_csv('./output.csv')['default_prob'])

0.8951198290471752

In [15]:
import pandas as pd
import datetime
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import mstats
import random
import lightgbm as lgb
import gc
import multiprocessing 
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

In [ ]:
df = pd.read_csv('../input/d/luyangshang/finance/data.csv')
df = df.drop(columns = ['Unnamed: 0'])

In [ ]:
id_lst = df.index.unique().tolist()
random.shuffle(id_lst)
train_id = id_lst[:int(len(id_lst)*0.7)]
test_id = id_lst[int(len(id_lst)*0.7):]
df.loc[test_id]

In [ ]:
df = df.set_index('id')

In [ ]:
id_lst = df.index.unique().tolist()[:int(len(id_lst)*0.7)]
random.shuffle(id_lst)
train_id = id_lst[:int(len(id_lst)*0.7)]
test_id = id_lst[int(len(id_lst)*0.7):]
train = df.loc[train_id]
test = df.loc[test_id]

In [ ]:
feature

In [ ]:
# 删掉无用特征
feature = [x for x in train.columns]
feature = list(set(feature))
feature.remove('defaulted')
print(len(feature))

In [ ]:
len(id_lst)

In [ ]:
id_lst = pd.Series(id_lst)
id_lst

In [ ]:
params = {'num_leaves': 63, #结果对最终效果影响较大，越大值越好，太大会出现过拟合
          'objective': 'binary', #定义的目标函数
          'max_depth': -1,
          'learning_rate': 0.1,
          "min_sum_hessian_in_leaf": 6,
          "boosting": "gbdt",
          "feature_fraction": 1,  #提取的特征比率
          "bagging_freq": 1,
          "bagging_fraction": 1,
          "bagging_seed": 11,
#           "lambda_l1": 0.1,             #l1正则
          # 'lambda_l2': 0.001,     #l2正则
          "verbosity": -1,
          "nthread": multiprocessing.cpu_count(),                #线程数量，-1表示全部线程，线程越多，运行的速度越快
          'metric': {'binary_logloss', 'auc'},  ##评价函数选择
          "random_state": 2077, #随机数种子，可以防止每次运行的结果不一致
          # 'device': 'gpu' ##如果安装的事gpu版本的lightgbm,可以加快运算
          }

n_fold = 5
# oof_pre = np.zeros(len(train))
y_pre = np.zeros(len(test))

kf = KFold(n_splits=5)
for train_index, test_index in kf.split(id_lst):
    train_id, val_id = id_lst[train_index], id_lst[test_index]


    
    trn_data = lgb.Dataset(train[feature].loc[train_id], label=train['defaulted'].loc[train_id])
    val_data = lgb.Dataset(train[feature].loc[val_id], label=train['defaulted'].loc[val_id])

    clf = lgb.train(params,
                    trn_data,
                    2000,
                    valid_sets=[trn_data, val_data],
                    verbose_eval=50,
                    early_stopping_rounds=50)
    # out of fold
    # oof_pre[val_id] = clf.predict(val_data[feature], num_iteration=clf.best_iteration)

    y_pre += clf.predict(test[feature], num_iteration=clf.best_iteration) / n_fold

In [ ]:
roc_auc_score(test['defaulted'], y_pre)

In [ ]:
pd.Series(y_pre).hist()

In [ ]:
fea_imp = pd.DataFrame()
fea_imp['imp'] = clf.feature_importance()
fea_imp['fea_name'] = clf.feature_name()
fea_imp = fea_imp.sort_values('imp')[:30]
plt.figure(figsize=(10, 10))
plt.barh(fea_imp['fea_name'],fea_imp['imp'],height =0.5)